# NEAT (NeuroEvolution of Augmenting Topologies)

In [200]:
import neat
import numpy as np
import pickle
from sklearn.metrics import f1_score, classification_report

In [201]:
def evaluate_genome(genomes, config, X, y):
    X = np.asarray(X).reshape(-1, 4)
    y = np.asarray(y).flatten()
    
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        predictions = []
        
        for xi in X:
            output = net.activate(xi)
            predictions.append(np.argmax(output))
        
        f1 = f1_score(y, predictions, average='weighted')

        genome.fitness = f1

def train_neat(X_train, y_train, config_path, save_as):
    print("Форма данных:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )
    
    population = neat.Population(config)
    population.add_reporter(neat.StdOutReporter(True))
    population.add_reporter(neat.StatisticsReporter())
    
    evaluator = lambda genomes, cfg: evaluate_genome(genomes, cfg, X_train, y_train)
    winner = population.run(evaluator, 50)
    
    with open(save_as, 'wb') as f:
        pickle.dump(winner, f)


In [202]:
X_train = np.load('../X_train.npy')
X_test  = np.load('../X_test.npy')
y_train = np.load('../y_train.npy')
y_test  = np.load('../y_test.npy')
sample1 = 1_000
sample2= 10_000


In [203]:
print('model1')
train_neat(X_train[sample1], y_train[sample1],
        config_path='neat_config.cfg',
        save_as='neat_model1.pkl')

model1
Форма данных:
X_train: (4,), y_train: ()

 ****** Running generation 0 ****** 

Population's average fitness: 1.00000 stdev: 0.00000
Best fitness: 1.00000 - size: (1, 4) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.098, standard deviation 0.428
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      1.0    0.000     0
Total extinctions: 0
Generation time: 0.091 sec

 ****** Running generation 1 ****** 

Population's average fitness: 1.00000 stdev: 0.00000
Best fitness: 1.00000 - size: (1, 4) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.464, standard deviation 0.425
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      1.0    0.000     1
Total extinctions: 0
Generation time: 0.080 sec (0.085 average)

 ****** Running generation 2 ****** 

P

In [204]:
print('model2')
train_neat(X_train[sample2], y_train[sample2],
        config_path='neat_config.cfg',
        save_as='neat_model2.pkl')

model2
Форма данных:
X_train: (4,), y_train: ()

 ****** Running generation 0 ****** 

Population's average fitness: 1.00000 stdev: 0.00000
Best fitness: 1.00000 - size: (1, 4) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.040, standard deviation 0.353
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      1.0    0.000     0
Total extinctions: 0
Generation time: 0.079 sec

 ****** Running generation 1 ****** 

Population's average fitness: 1.00000 stdev: 0.00000
Best fitness: 1.00000 - size: (1, 4) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.288, standard deviation 0.417
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      1.0    0.000     1
Total extinctions: 0
Generation time: 0.076 sec (0.077 average)

 ****** Running generation 2 ****** 

P

In [205]:
def load_and_test_model(model_path, config_path, X_test, y_test):
    with open(model_path, 'rb') as f:
        genome = pickle.load(f)
    
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        config_path)
    
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    
    X_test = np.asarray(X_test).reshape(-1, 4)
    
    predictions = []
    for xi in X_test:
        xi = np.asarray(xi).flatten()
        output = net.activate(xi)    
        predictions.append(np.argmax(output))
    
    y_true = y_test.flatten()
    y_pred = np.array(predictions)
    
    print("\nТестовые метрики:")
    print(classification_report(y_true, y_pred))
    print(f"F1-score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    
    return y_pred

In [206]:
predictions = load_and_test_model(
    model_path='neat_model1.pkl',
    config_path='neat_config.cfg',
    X_test=X_test[120_000],
    y_test=y_test[120_000]
)


Тестовые метрики:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

F1-score: 1.0000


In [207]:
predictions = load_and_test_model(
    model_path='neat_model2.pkl',
    config_path='neat_config.cfg',
    X_test=X_test[100_000],
    y_test=y_test[100_000]
)


Тестовые метрики:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

F1-score: 1.0000
